# transaction items dataset builder

We want to build the dataset we will use in order to give people the recommended product.  
We also analyse and clean data.

Read data

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from pandas_profiling import ProfileReport
import plotly.express as px


df = pd.read_csv("../data/KaDo.csv")
df

Check the following:
- amilly analysis
- na values
- null values

In [ ]:
px.histogram(df, y="FAMILLE", title="before cleaning").update_yaxes(categoryorder="total ascending")

In [ ]:
print(f"""
shape: {df.shape}
na: {df.isna().sum().sum()}
null: {df.isnull().sum().sum()}
dropped: {len(df[df["FAMILLE"].isin(["MULTI FAMILLES", "SANTE NATURELLE"])])}
""")

mask = ~df["FAMILLE"].isin(["MULTI FAMILLES","SANTE NATURELLE"])
df = df[mask]
px.histogram(df, y="FAMILLE", title="after cleaning").update_yaxes(categoryorder="total ascending")

g et each product as index and its associated FAMILLE/MAILLE/UNIVERS.

In [ ]:
obj_df = df.groupby('LIBELLE').agg({
    'FAMILLE': lambda x: x.value_counts().index[0],
    'MAILLE': lambda x: x.value_counts().index[0],
    'UNIVERS': lambda x: x.value_counts().index[0]
}).reset_index()

obj_df.set_index('LIBELLE', inplace=True)
obj_df

get median price for each product because we have a lot of weird prices and hight variance.

In [ ]:
obj_df['PRIX_NET_MEDIAN'] = df.groupby('LIBELLE')['PRIX_NET'].median()
obj_df

In [ ]:
px.histogram(df, y="FAMILLE")

In [ ]:
sns.histplot(obj_df['PRIX_NET_MEDIAN'], kde=True)

In [ ]:
obj_df[["MAILLE", "UNIVERS", "FAMILLE"]].describe()

## Profiling report

The idea is to analyse the variables and their relations through metrics.


In [ ]:
profile = ProfileReport(
    obj_df,
    orange_mode=True,
    sortby="PRIX_NET_MEDIAN",
    title="transaction items report"
)
profile.to_file("transaction_items_eda_report.html")
obj_df.to_csv("transaction_items.csv")